## Example 🖼️
| Thumbnail | Video (*± Middle Frame*) |
|:---:|:---:|
| <img src="https://raw.githubusercontent.com/kubinka0505/YTMVG/master/Documents/Pictures/Examples/Google_Colab/Thumbnail.jpg" height=200> | <img src= "https://raw.githubusercontent.com/kubinka0505/YTMVG/master/Documents/Pictures/Examples/Google_Colab/Cover.png" height=200> |
| *Subjectively basing on the image clusters, a<br>16:9 background is generated, onto the<br>center of which the image is placed.* | *Generates the video with dimensions that<br>match the input image, and appends<br>the specified audio to it.*|

---
<br>

### 📱 **Information for mobile users**
<font color="red"><b>If cells don't output anything and are finished immediately, please re-run the site in the <code>Desktop site</code> mode</b></font> (Chrome)

<br>

---
<br>

[![](https://img.shields.io/badge/github-open-lightgray?logo=github&logoColor=white&style=for-the-badge)](https://github.com/kubinka0505/YTMVG)

In [ ]:
#@title # Download requirements 📥

import os

#-=-=-=-#

!rm -rf YTMVG
!git clone https://github.com/kubinka0505/YTMVG

os.chdir("YTMVG")
!apt-get install -qq -o=Dpkg::Use-Pty=0 jpegoptim
print()
%pip install -r requirements.txt | grep -v "already satisfied"

# Processing ⚙️

In [ ]:
#@title # <font color=lime>**1.**</font> Setup ⚙️

import os, warnings
from pathlib import Path
from zipfile import ZipFile
from google.colab import files
warnings.filterwarnings("ignore")

#-=-=-=-#

if os.path.exists("YTMVG"):
	os.chdir("YTMVG")
else:
	raise SystemExit("Requirements were not installed.\nPlease run the previous cell.")

Audio_URL = "" #@param {type: "string"}
Image_URL = "" #@param {type: "string"}


#@markdown ---
#@markdown # Thumbnail *(background color)* 🖼️

Type = "Wet" #@param ["Wet", "Dry"]
Brightness = 40 #@param {type: "slider", min: 25, max: 75, step: 0.1}
Saturation = 100 #@param {type: "slider", min: 75, max: 250, step: 0.1}
Resize = True #@param {type: "boolean"}
Generate_Thumbnail = True #@param {type: "boolean"}

#@markdown <br>

#@markdown # Video 📹
Maximum_Resolution = "1080p" #@param ["4320p", "2160p", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"]
WebM = False #@param {type: "boolean"}
Generate_Video = True #@param {type: "boolean"}

#@markdown <br>

#@markdown # Audio 🔊
Bitrate = 255 #@param {type: "slider", min: 96, max: 256, step: 1}

#@markdown <br>

#@markdown # Desciption 📝
Generate_Description = True #@param {type: "boolean"}

#@markdown ---
#@markdown # Program ⚙️
Debug = False #@param {type: "boolean"}
Verbose = False #@param {type: "boolean"}
Help = False #@param {type: "boolean"}

#-=-=-=-#

if not Debug:
	if not Audio_URL:
		raise SystemExit("No audio URL was provided.")

#-=-=-=-#

# URLs
os.environ["AUDIO"] = Audio_URL
os.environ["IMAGE"] = f'-i "{Image_URL}"' if Image_URL else ""

# Thumbnail
os.environ["COLOR"] = "1" if Type.lower() == "dry" else "2"
os.environ["BRIGHTNESS"] = str(round(Brightness / 1, 2))
os.environ["SATURATION"] = str(round(Saturation / 1, 2))
os.environ["RESIZE"] = "-rt" if Resize else ""
os.environ["GEN_THUMB"] = "" if Generate_Thumbnail else "-nt"

# Video
os.environ["RESOLUTION"] = Maximum_Resolution.strip("p") + "p"
os.environ["WEBM"] = "-webm" if WebM else ""
os.environ["GEN_VIDEO"] = "" if Generate_Video else "-nv"

# Audio
os.environ["BITRATE"] = str(Bitrate)

# Description
os.environ["GEN_DESC"] = "" if Generate_Description else "-nd"

# Program
os.environ["EXEC"] = "echo " if Debug else os.sys.executable
os.environ["VERBOSE"] = "-v 2" if Verbose else ""
os.environ["HELP"] = "-h" if Help else ""

In [ ]:
#@title # <font color=gold>**2.**</font> Run ▶️

!$EXEC "YTMVG.py" \
	-a "$AUDIO" -ab $BITRATE\k \
	$IMAGE -r $RESOLUTION \
	-c $COLOR $RESIZE \
	-bgb $BRIGHTNESS\% \
	-bgs $SATURATION\% \
	$WEBM \
	$GEN_THUMB $GEN_VIDEO $GEN_DESC \
	$VERBOSE -nod $HELP

#-=-=-=-#

os.chdir("..")

In [ ]:
#@title # <font color=lime></font> Download archive with files *(optional)* 📥

from google.colab import files

Latest_Directory = [str(Directory.resolve()) for Directory in Path("YTMVG/Videos").glob("*")]
Latest_Directory = sorted(Latest_Directory, key = os.path.getmtime)[-1]

Archive_Name = "_".join(("YTMVG", os.path.basename(Latest_Directory)))
Archive_Name = Archive_Name.replace(" ", "_") + ".zip"

#-=-=-=-#

# Archive Creation
print("Making Archive...")
with ZipFile(Archive_Name, "w", 14, 1, 9) as Archive:
	for File in Path(Latest_Directory).rglob("*"):
		File = str(File.resolve())

		print('\tWriting "{0}"...'.format(
			File.split(os.getcwd())[-1])
		)
		Archive.write(File, os.path.basename(File))

files.download(Archive_Name)

#-=-=-=-#

print("\n" + "─" * 32 + "\n\nDone!")